## Dataset Description

**Name:** NBA Players Database

**Source:** [Kaggle - NBA Players Database by Mustafa Yağız Fırat](https://www.kaggle.com/datasets/yagizfiratt/nba-players-database)

**Summary:**  
This dataset provides comprehensive information on NBA players, including biographical details (such as name, birthdate, nationality, and college), physical attributes (height, weight), and career data (draft year, team, position, and more).

**Structure:**  
- Each row represents a unique NBA player or a player-season, depending on the STATS_TIMEFRAME column.
- The dataset contains 5025 rows and 26 columns.
- The dataset contains the following columns:

| Column Name   | Description                                      |
|---------------|--------------------------------------------------|
| PERSON_ID	| Unique identifier for each NBA player |
| PLAYER_LAST_NAME | Player’s last name |
| PLAYER_FIRST_NAME |	Player’s first name |
| PLAYER_SLUG | URL-friendly version of the player’s full name (often used for web links) |
| TEAM_ID	| Unique identifier for the team |
| TEAM_SLUG | URL-friendly version of the team name |
| IS_DEFUNCT | Indicates if the team is defunct (0 = active, 1 = defunct) |
| TEAM_CITY | City where the team is based |
| TEAM_NAME | Name of the NBA team |
| TEAM_ABBREVIATION | Standard abbreviation for the team name (e.g., LAL for Los Angeles Lakers) |
| JERSEY_NUMBER | Player’s jersey number |
| POSITION | Player’s position (e.g., G = Guard, F = Forward, C = Center, F-G = Forward-Guard) |
| HEIGHT | Player’s height (in feet-inches format, e.g., 6-10) |
| WEIGHT | Player’s weight (in pounds) |
| COLLEGE | College attended by the player |
| COUNTRY | Country of origin (nationality) |
| DRAFT_YEAR | The year the player was drafted into the NBA |
| DRAFT_ROUND | The round in which the player was selected during the NBA draft |
| DRAFT_NUMBER | The overall pick number of the player in the NBA draft |
| ROSTER_STATUS | Indicates if the player is currently on a roster (1.0 = active, 0.0 = inactive/retired) |
| PTS | Points per game (or per season/career, depending on STATS_TIMEFRAME) |
| REB | Rebounds per game (or per season/career) |
| AST | Assists per game (or per season/career) |
| STATS_TIMEFRAME | Indicates whether the stats are for the player's entire career ("Career") or a single season ("Season") |
| FROM_YEAR | The first year of the player's NBA career or the season covered by the stats |
| TO_YEAR | The last year of the player's NBA career or the season covered by the stats |

**Provenance:**  
Compiled and published by Mustafa Yağız Fırat on Kaggle. Last updated in 2025.

**License:**  
Check the Kaggle page for licensing details; the dataset is typically available for educational and non-commercial use.

**Note:**  
- The TEAM_NAME column refers to the team the player last played for.
- The dataset was likely compiled by scraping player information from the [NBA's official website](https://www.nba.com/players) and aggregating it into a structured CSV file.

**Potential Implications and Biases:**
- Since the data was collected from the NBA’s official website, it is likely accurate for players who are currently active or recently retired. However, historical players or those with incomplete records may not be fully represented.
- The web scraping process may introduce errors if the website structure changed or if certain fields were missing at the time of collection.
- Career and season statistics may not be directly comparable across different eras due to changes in the game, league rules, and data recording methods.




In [11]:
import pandas as pd
players_df = pd.read_csv('PlayerIndex_nba_stats.csv')
players_df.shape

(5025, 26)

**The dataset contains 5025 observations through the pandas 'shape' attribute.**